In [1]:
# Notebook for fine-tuning BERT related models

In [2]:
from math import sqrt
import regex as re
import os
from glob import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error
import torch
import transformers as ppb
import warnings

warnings.filterwarnings('ignore')

BERT EMBEDDER
+ TIME OF EACH SENT

LSTM (WITH ATTENTION) OVER SEQUENCE ??? 

In [3]:
os.getcwd()

'/Users/tom/phd/ADReSS_Challenge/ADReSS_Challenge'

In [8]:
pitt_cookie_cc = '../data/Pitt/Control/cookie/**'
pitt_fluency_cc = '../data/Pitt/Control/fluency/**'
pitt_cookie_ad = '../data/Pitt/Dementia/cookie/**'
pitt_fluency_ad = '../data/Pitt/Dementia/fluency/**'
pitt_recall_ad = '../data/Pitt/Dementia/recall/**'
pitt_sentence_ad = '../data/Pitt/Dementia/sentence/**'
ad_datas = [pitt_cookie_ad, pitt_fluency_ad, pitt_recall_ad, pitt_sentence_ad]
ctrl_datas = [pitt_cookie_cc, pitt_fluency_cc]

In [75]:
def extract_data(file_name):
    par = {}
    par['id'] = file_name.split('/')[-1].split('.cha')[0]
    f = iter(open(file_name))
    l = next(f)
    speech = []
    try:
        curr_speech = ''
        while (True):
            if l.startswith('@ID'):
                participant = [i.strip() for i in l.split('|')]
                if participant[2] == 'PAR':
                    par['mmse'] = '' if len(participant[8]) == 0 else float(participant[8])
                    par['sex'] = participant[4][0] if len(participant[4]) else 'n/a'
                    age = participant[3].replace(';', '')
                    
                    try:
                        par['age'] = int(float(age)) if len(age) > 0 else 'n/a'
                    except:
                        print(participant)
                        print(age)
            if l.startswith('*PAR:') or l.startswith('*INV'):
                curr_speech = l
            elif len(curr_speech) != 0 and not(l.startswith('%') or l.startswith('*')):
                curr_speech += l
            elif len(curr_speech) > 0:
                speech.append(curr_speech)
                curr_speech = ''
            l = next(f)
    except StopIteration:
        pass

    clean_par_speech = []
    clean_all_speech = []
    speech_time_segments = []
    is_par = False
    for s in speech:
        def _clean(s):
            try:
                speech_time_segments.append([*map(int, re.search('\x15(\d*_\d*)\x15', s).groups()[0].split('_'))])
            except:
                speech_time_segments.append([])
            s = re.sub('\x15\d*_\d*\x15', '', s) # remove time block 
            s = re.sub('\[.*\]', '', s) # remove other speech artifacts [.*]
            s = s.strip()
            s = re.sub('\t|\n|<|>', '', s) # remove tab, new lines, inferred speech??, ampersand, &
            return s
        
        if s.startswith('*PAR:'):
            is_par = True
        elif s.startswith('*INV:'):
            is_par = False
            s = re.sub('\*INV:\t', '', s) # remove prefix
        if is_par:
            s = re.sub('\*PAR:\t', '', s) # remove prefix    
            clean_par_speech.append(_clean(s))
        clean_all_speech.append(_clean(s))
    
    par['speech'] = speech
    par['clean_speech'] = clean_all_speech
    par['clean_par_speech'] = clean_par_speech
    par['joined_all_speech'] = ' '.join(clean_all_speech)
    par['joined_all_par_speech'] = ' '.join(clean_par_speech)
    
    # sentence times
#     par['per_sent_times'] = [speech_time_segments[i][1] - speech_time_segments[i][0] if len(speech_time_segments[i]) else -1
#                              for i in range(len(speech_time_segments))]
#     par['total_time'] =  speech_time_segments[-1][1] - speech_time_segments[0][0]
#     par['time_before_par_speech'] = speech_time_segments[0][0]
#     par['time_between_sents'] = [0 if i == 0 else max(0, speech_time_segments[i][0] - speech_time_segments[i-1][1]) 
#                                  if len(speech_time_segments[i]) else -1
#                                  for i in range(len(speech_time_segments))]
    return par

In [76]:
def parse_data(data_dir, ad=True):    
    df = pd.DataFrame([extract_data(fn) for fn in glob(data_dir)])
    df['ad'] = 1 if ad else 0
    df = df.sample(frac=1).reset_index(drop=True)
    return df

In [77]:
ad_df = pd.concat([parse_data(ad_dir) for ad_dir in ad_datas])

In [81]:
ctrl_df = pd.concat([parse_data(ctrl_dir) for ctrl_dir in ctrl_datas])

In [84]:
ad_df.sex.value_counts()

f      620
m      427
n/a      2
Name: sex, dtype: int64

In [87]:
ctrl_df.shape

(244, 10)

In [88]:
ad_df.shape

(1049, 10)

In [90]:
ad_df.joined_df

AttributeError: 'DataFrame' object has no attribute 'joined_df'